In [1]:
import pandas as pd
import folium
import re
import numpy as np


# Sources:
# https://www.bikeshare.com/data/
# https://www.bluebikes.com/system-data
# https://crashviewer.nhtsa.dot.gov/CrashAPI
# https://hifld-geoplatform.opendata.arcgis.com/datasets/hospitals/data?selectedAttribute=BEDS Hospital Data
# https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population Cities
# https://umich-edu-2.domo.com/invite/58636bdf-056a-484c-ac97-53af091b73a3
import warnings;
warnings.filterwarnings('ignore');


In [2]:
#Load Bike trips and stations data
bike_df = pd.read_csv("data/202009-bluebikes-tripdata.csv")
bike_stations_df = pd.read_csv("data/current_bluebikes_stations.csv", header=1)

#Load and combine FARS data (Fatality)
crash_2017_df = pd.read_csv('data/Accidents_2017.csv')
crash_2018_df = pd.read_csv('data/Accidents_2018.csv')
crash_df = crash_2017_df.append(crash_2018_df)

#Load Hospital data
hospitals_df = pd.read_csv('data/Hospitals.csv')

#Load Top10 Cities data
cities_df = pd.read_csv('data/Cities.csv')

#Load Safety Ratings
safety_ratings = pd.read_csv('data/NHTSA Safety Rating.csv')

In [3]:
# So the dataset is retrieved from: https://crashviewer.nhtsa.dot.gov/CrashAPI
#GET FARS data from accident, pbtype, and vehicle tables 
fars_acc2018 = pd.read_csv('data/FARS/FARS2018/ACCIDENT.csv')
fars_acc2017 = pd.read_csv('data/FARS/FARS2017/ACCIDENT.csv')
fars_acc2016 = pd.read_csv('data/FARS/FARS2016/ACCIDENT.csv')

fars_PB2018 = pd.read_csv('data/FARS/FARS2018/PBTYPE.csv')
fars_PB2017 = pd.read_csv('data/FARS/FARS2017/PBTYPE.csv')
fars_PB2016 = pd.read_csv('data/FARS/FARS2016/PBTYPE.csv')

fars_veh2018 = pd.read_csv('data/FARS/FARS2018/VEHICLE.csv',encoding= 'unicode_escape')
fars_veh2017 = pd.read_csv('data/FARS/FARS2017/VEHICLE.csv',encoding= 'unicode_escape')
fars_veh2016 = pd.read_csv('data/FARS/FARS2016/VEHICLE.csv',encoding= 'unicode_escape')

#Join FARS accident and vehicle tables on state and st_case
fars_acc_veh_2018 = pd.merge(fars_acc2018, fars_veh2018,  how='left', left_on=['STATE','ST_CASE'], right_on = ['STATE','ST_CASE'])
fars_acc_veh_2017 = pd.merge(fars_acc2017, fars_veh2017,  how='left', left_on=['STATE','ST_CASE'], right_on = ['STATE','ST_CASE'])
fars_acc_veh_2016 = pd.merge(fars_acc2016, fars_veh2016,  how='left', left_on=['STATE','ST_CASE'], right_on = ['STATE','ST_CASE'])

#Join FARS accident and vehicle tables on state and st_case
fars_all_2018 = pd.merge(fars_acc_veh_2018, fars_PB2018,  how='left', left_on=['STATE','ST_CASE'], right_on = ['STATE','ST_CASE'])
fars_all_2017 = pd.merge(fars_acc_veh_2017, fars_PB2017,  how='left', left_on=['STATE','ST_CASE'], right_on = ['STATE','ST_CASE'])
fars_all_2016 = pd.merge(fars_acc_veh_2016, fars_PB2016,  how='left', left_on=['STATE','ST_CASE'], right_on = ['STATE','ST_CASE'])

FARS_16_17_18 = fars_all_2018.append([fars_all_2017, fars_all_2016]).reset_index()

In [4]:
#GET CRSS data from accident, pbtype, and vehicle tables 
crss_acc2018 = pd.read_csv('data/CRSS/CRSS2018/ACCIDENT.csv')
crss_acc2017 = pd.read_csv('data/CRSS/CRSS2017/ACCIDENT.csv')
crss_acc2016 = pd.read_csv('data/CRSS/CRSS2016/ACCIDENT.csv')

crss_PB2018 = pd.read_csv('data/CRSS/CRSS2018/PBTYPE.csv')
crss_PB2017 = pd.read_csv('data/CRSS/CRSS2017/PBTYPE.csv')
crss_PB2016 = pd.read_csv('data/CRSS/CRSS2016/PBTYPE.csv')

crss_veh2018 = pd.read_csv('data/CRSS/CRSS2018/VEHICLE.csv',encoding= 'unicode_escape')
crss_veh2017 = pd.read_csv('data/CRSS/CRSS2017/VEHICLE.csv',encoding= 'unicode_escape')
crss_veh2016 = pd.read_csv('data/CRSS/CRSS2016/VEHICLE.csv',encoding= 'unicode_escape')

#Join CRSS accident and vehicle tables on state and st_case
crss_acc_veh_2018 = pd.merge(crss_acc2018, crss_veh2018,  how='left', left_on=['CASENUM'], right_on = ['CASENUM'])
crss_acc_veh_2017 = pd.merge(crss_acc2017, crss_veh2017,  how='left', left_on=['CASENUM'], right_on = ['CASENUM'])
crss_acc_veh_2016 = pd.merge(crss_acc2016, crss_veh2016,  how='left', left_on=['CASENUM'], right_on = ['CASENUM'])

crss_all_2018 = pd.merge(crss_acc_veh_2018, crss_PB2018,  how='left', left_on=['CASENUM','VEH_NO'], right_on = ['CASENUM','VEH_NO'])
crss_all_2017 = pd.merge(crss_acc_veh_2017, crss_PB2017,  how='left', left_on=['CASENUM','VEH_NO'], right_on = ['CASENUM','VEH_NO'])
crss_all_2016 = pd.merge(crss_acc_veh_2016, crss_PB2016,  how='left', left_on=['CASENUM','VEH_NO'], right_on = ['CASENUM','VEH_NO'])

CRSS_16_17_18 = crss_all_2018.append([crss_all_2017, crss_all_2016]).reset_index()



In [5]:
# Select the columns to use
safety_ratings = safety_ratings[['OverallRating','OverallFrontCrashRating','FrontCrashDriversideRating',
                                 'FrontCrashPassengersideRating','OverallSideCrashRating',
                                 'SideCrashDriversideRating','SideCrashPassengersideRating','RolloverRating',
                                 'RolloverRating2','RolloverPossibility','RolloverPossibility2',
                                 'SidePoleCrashRating','ComplaintsCount','RecallsCount','InvestigationCount',
                                 'ModelYear','Make','Model','VehicleDescription','VehicleId',
                                 'NHTSAForwardCollisionWarning','NHTSALaneDepartureWarning']]

In [6]:
# cities_df = pd.read_csv('data/Cities.csv')
#Extract Lat and Long independently to plot cities and convert to float
cities_df['Lat'] = cities_df['Location'].str.extract('(\d+\.\d+)').astype(float)
cities_df['Lon'] = (cities_df['Location'].str.extract('\s(\d+\.\d+)').astype(float))*-1

#Convert Lat and Long to radians to faciliate creating a radius for the city
cities_df['Lat_rad'] = cities_df['Lat'] * np.pi / 180
cities_df['Lon_rad'] = cities_df['Lon'] * np.pi / 180
# Get land area in km2
cities_df['Land'] = cities_df['2016 land area [km2]'].str.extract('([\d,]+\.\d+)').replace(',','')
cities_df['Land'] = cities_df['Land'].str.replace(',','').astype(float)
#Clean City name
cities_df['City'] = cities_df['City'].str.replace(r"\[.*\]", '')
#Remove commas and units for numerical values
cities_df['Estimate 2019'] = cities_df['Estimate 2019'].str.replace(',','').astype(float)
cities_df['Census 2010'] = cities_df['Census 2010'].str.replace(',','').astype(float)

In [ ]:
cities_df

In [7]:
#Convert Lat and Long to radians in the FARS data to extract fatalities in a radius for top ten cities
FARS_16_17_18['Lat_rad'] = FARS_16_17_18['LATITUDE'] * np.pi / 180
FARS_16_17_18['Lon_rad'] = FARS_16_17_18['LONGITUD'] * np.pi / 180

def radius(Lat1, Lon1, Lat2, Lon2):
    # http://www.movable-type.co.uk/scripts/latlong.html?from=48.6093070,-122.4259880&to=48.5928360,-122.4216130
    # http://janmatuschek.de/LatitudeLongitudeBoundingCoordinates
    # Lat/Lon1 is for the top city and Lat/Lon2 is for the accident in FARS data set
    return np.arccos((np.sin(Lat1) * np.sin(Lat2)) + 
                     (np.cos(Lat1) * np.cos(Lat2) * np.cos(Lon2 - (Lon1)))) * 6371

rad = np.sqrt(cities_df['Land'][0])

FARS_Top_Cities = pd.DataFrame() 
 
# for loop to create FARS_Top_Cities df with the fatalities of each city based on a radius and location 
for i in range(len(cities_df)):
    # Calculate the radius for each city based on the land of the city
    rad = np.sqrt(cities_df['Land'][i])
    
    #Extract the fatalities from FARS based on radius distance of the city
    city_df = pd.DataFrame()
    city_df = FARS_16_17_18[radius(cities_df['Lat_rad'][i],cities_df['Lon_rad'][i],
                                   FARS_16_17_18['Lat_rad'],FARS_16_17_18['Lon_rad']) <= rad]
    
    # Label each record and rank with the city it is on
    city_df['City'] = cities_df['City'][i]
    city_df['Rank'] = i+1
    
    # Append result to FARS_Top_Cities df
    FARS_Top_Cities = FARS_Top_Cities.append(city_df)
    
    
    
FARS_Top_Cities.reset_index(inplace=True)

In [8]:
# Add total fatalities to cities_df to perform some analysis per habitants
tot_accidents = FARS_Top_Cities.groupby('City').count()
tot_accidents[['index']]
cities_df = pd.merge(cities_df, tot_accidents[['index']], on='City')
cities_df.rename(columns = {'index':'Total_Fatal'}, inplace = True)

In [9]:
cities_df['Fatalities_per_Habitant'] = cities_df['Total_Fatal']/cities_df['Estimate 2019']*10000
cities_df['Fatalities_every_Habitant'] = cities_df['Estimate 2019']/cities_df['Total_Fatal']
cities_df

,Rank 2019,City,State,Estimate 2019,Census 2010,Change,2016 land area [sq mi],2016 land area [km2],2016 population density [sq mi],2016 population density [km2],Location,Lat,Lon,Lat_rad,Lon_rad,Land,Total_Fatal,Fatalities_per_Habitant,Fatalities_every_Habitant
0,1,New York City,New York,8336817.0,8175133.0,1.98%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",40.6635°N 73.9387°W,40.6635,-73.9387,0.709712,-1.290474,780.9,1348,1.616924,6184.582344
1,2,Los Angeles,California,3979576.0,3792621.0,4.93%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34.0194°N 118.4108°W,34.0194,-118.4108,0.593751,-2.066658,1213.9,2409,6.053409,1651.961810
2,3,Chicago,Illinois,2693976.0,2695598.0,−0.06%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41.8376°N 87.6818°W,41.8376,-87.6818,0.730204,-1.530336,588.7,924,3.429875,2915.558442
3,4,Houston,Texas,2320268.0,2100263.0,10.48%,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",29.7866°N 95.3909°W,29.7866,-95.3909,0.519874,-1.664885,1651.1,2048,8.826567,1132.943359
4,5,Phoenix,Arizona,1680992.0,1445632.0,16.28%,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",33.5722°N 112.0901°W,33.5722,-112.0901,0.585945,-1.956341,1340.6,1785,10.618730,941.732213
5,6,Philadelphia,Pennsylvania,1584064.0,1526006.0,3.80%,134.2 sq mi,347.6 km2,"11,683/sq mi","4,511/km2",40.0094°N 75.1333°W,40.0094,-75.1333,0.698296,-1.311323,347.6,691,4.362197,2292.422576
6,7,San Antonio,Texas,1547253.0,1327407.0,16.56%,461.0 sq mi,"1,194.0 km2","3,238/sq mi","1,250/km2",29.4724°N 98.5251°W,29.4724,-98.5251,0.514390,-1.719587,1194.0,864,5.584090,1790.802083
7,8,San Diego,California,1423851.0,1307402.0,8.91%,325.2 sq mi,842.3 km2,"4,325/sq mi","1,670/km2",32.8153°N 117.1350°W,32.8153,-117.1350,0.572735,-2.044391,842.3,627,4.403551,2270.894737
8,9,Dallas,Texas,1343573.0,1197816.0,12.17%,340.9 sq mi,882.9 km2,"3,866/sq mi","1,493/km2",32.7933°N 96.7665°W,32.7933,-96.7665,0.572351,-1.688894,882.9,1520,11.313118,883.929605
9,10,San Jose,California,1021795.0,945942.0,8.02%,177.5 sq mi,459.7 km2,"5,777/sq mi","2,231/km2",37.2967°N 121.8189°W,37.2967,-121.8189,0.650950,-2.126141,459.7,353,3.454705,2894.603399


In [ ]:
# Create Folium Map of Top ten cities and add different attributes to the map such as Bike Stations, Accidents, Hospitals...
m = folium.Map(location=[40, -96], zoom_start=4.4,prefer_canvas=True)

for i in range(len(cities_df)): #Top Ten Cities
    folium.Circle(location=[cities_df['Lat'][i], cities_df['Lon'][i]],
      popup='name',
      radius=15,
      color='blue',
      fill=True,
      fill_color='blue'
   ).add_to(m)
    
for i in range(len(FARS_Top_Cities)): #Crash
    folium.Circle(location=[FARS_Top_Cities['LATITUDE'][i], FARS_Top_Cities['LONGITUD'][i]],
      popup='name',
      radius=80,
      color='red',
      fill=True,
      fill_color='red'
   ).add_to(m)    
    
# for i in range(len(FARS_16_17_18)): #Crash invovled cyclist
#     folium.Circle(location=[FARS_16_17_18['LATITUDE'][i], FARS_16_17_18['LONGITUD'][i]],
#       popup='name',
#       radius=10,
#       color='yellow',
#       fill=True,
#       fill_color='yellow'
#    ).add_to(m)   
    
# for i in range(len(hospitals_df)): #Hospital Location
#     folium.Circle(location=[hospitals_df['LATITUDE'][i], hospitals_df['LONGITUDE'][i]],
#       popup='name',
#       radius=50,
#       color='green',
#       fill=True,
#       fill_color='green'
#    ).add_to(m) 

m

In [10]:
safety_ratings

,OverallRating,OverallFrontCrashRating,FrontCrashDriversideRating,FrontCrashPassengersideRating,OverallSideCrashRating,SideCrashDriversideRating,SideCrashPassengersideRating,RolloverRating,RolloverRating2,RolloverPossibility,...,ComplaintsCount,RecallsCount,InvestigationCount,ModelYear,Make,Model,VehicleDescription,VehicleId,NHTSAForwardCollisionWarning,NHTSALaneDepartureWarning
0,5,5,5,5,5,5,5,5,Not Rated,0.097,...,0.0,0.0,0.0,2019,ACURA,RLX,2019 Acura RLX 4 DR FWD,13009,NaN,NaN
1,5,5,5,5,5,5,5,5,Not Rated,0.098,...,0.0,0.0,0.0,2019,ACURA,TLX,2019 Acura TLX 4 DR AWD,13012,NaN,NaN
2,5,5,5,5,5,5,5,5,Not Rated,0.098,...,0.0,0.0,0.0,2019,ACURA,TLX,2019 Acura TLX 4 DR FWD,13011,NaN,NaN
3,5,4,4,4,5,5,5,4,Not Rated,0.111,...,0.0,0.0,0.0,2019,BUICK,CASCADA,2019 Buick Cascada C FWD,13017,NaN,NaN
4,5,5,5,5,5,5,5,4,Not Rated,0.179,...,0.0,0.0,0.0,2019,BUICK,ENCORE,2019 Buick Encore SUV AWD,13019,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6564,Not Rated,Not Rated,3,4,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,0.000,...,30.0,2.0,0.0,1990,TOYOTA,CELICA,1990 Toyota Celica 2-DR,2910,No,No
6565,Not Rated,Not Rated,3,2,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,0.000,...,28.0,1.0,0.0,1990,TOYOTA,COROLLA,1990 Toyota Corolla 4-DR,2880,No,No
6566,Not Rated,Not Rated,3,4,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,0.000,...,4.0,0.0,0.0,1990,TOYOTA,CRESSIDA,1990 Toyota Cressida 4-DR.,2855,No,No
6567,Not Rated,Not Rated,1,3,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,0.000,...,27.0,3.0,0.0,1990,TOYOTA,PICKUP,1990 Toyota Pickup 2-DR.,2907,No,No


In [12]:
list(CRSS_16_17_18.columns)

['index',
 'CASENUM',
 'REGION_x',
 'PSU_x',
 'PJ_x',
 'PSU_VAR_x',
 'URBANICITY_x',
 'STRATUM_x',
 'VE_TOTAL',
 'VE_FORMS_x',
 'PVH_INVL',
 'PEDS',
 'PERMVIT',
 'PERNOTMVIT',
 'NUM_INJ',
 'MONTH_x',
 'YEAR',
 'DAY_WEEK',
 'HOUR_x',
 'MINUTE_x',
 'HARM_EV_x',
 'ALCOHOL',
 'MAX_SEV',
 'MAN_COLL_x',
 'RELJCT1',
 'RELJCT2',
 'TYP_INT',
 'WRK_ZONE',
 'REL_ROAD',
 'LGT_COND',
 'WEATHER1',
 'WEATHER2',
 'WEATHER',
 'SCH_BUS',
 'INT_HWY',
 'CF1',
 'CF2',
 'CF3',
 'WKDY_IM',
 'HOUR_IM',
 'MINUTE_IM',
 'EVENT1_IM',
 'MANCOL_IM',
 'RELJCT1_IM',
 'RELJCT2_IM',
 'LGTCON_IM',
 'WEATHR_IM',
 'MAXSEV_IM',
 'NO_INJ_IM',
 'ALCHL_IM',
 'PSUSTRAT_x',
 'WEIGHT_x',
 'VEH_NO',
 'REGION_y',
 'PSU_y',
 'PJ_y',
 'PSU_VAR_y',
 'URBANICITY_y',
 'STRATUM_y',
 'VE_FORMS_y',
 'NUMOCCS',
 'MONTH_y',
 'HOUR_y',
 'MINUTE_y',
 'HARM_EV_y',
 'MAN_COLL_y',
 'UNITTYPE',
 'HIT_RUN',
 'MAKE',
 'MODEL',
 'BODY_TYP',
 'MOD_YEAR',
 'MAK_MOD',
 'VIN',
 'TOW_VEH',
 'J_KNIFE',
 'MCARR_I1',
 'MCARR_I2',
 'MCARR_ID',
 'GVWR',
 'V_C

In [13]:
CRSS_16_17_18

,index,CASENUM,REGION_x,PSU_x,PJ_x,PSU_VAR_x,URBANICITY_x,STRATUM_x,VE_TOTAL,VE_FORMS_x,...,PEDDIR,BIKEDIR,MOTDIR,MOTMAN,PEDLEG,PEDSNR,PEDCGP,BIKECGP,PSUSTRAT,WEIGHT
0,0,201800381338,4,56,208,208,1,9,4,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,201800381338,4,56,208,208,1,9,4,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,201800381338,4,56,208,208,1,9,4,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,201800381338,4,56,208,208,1,9,4,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,201800381366,4,56,214,214,1,7,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265874,82144,201600991811,4,41,163,41,1,10,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
265875,82145,201600991811,4,41,163,41,1,10,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
265876,82146,201600991821,4,41,160,41,1,8,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
265877,82147,201600991832,4,41,160,41,1,6,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
PEDS
MAX_SEV
MAXSEV_IM
NUM_INJ
NO_INJ_IM
NUMOCCS
MAKE
MODEL
MOD_YEAR
MDLYR_IM
ROLLOVER
ROLINLOC
IMPACT1
IMPACT1_IM
DEFORMED
M_HARM
VEVENT_IM
VEH_SC1
VEH_SC2
FIRE_EXP
MAX_VSEV
MXVSEV_IM
NUM_INJV
NUMINJ_IM
DR_SF1
ACC_TYPE